# MLB Pitch Type Analysis

## Overview

## Business Problem

In [26]:
# import packages

import numpy as np
import pandas as pd
from collections import defaultdict

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

import nltk
from nltk.probability import FreqDist
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import regexp_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

import matplotlib.pyplot as plt
import seaborn as sns
import string
import re

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

## Data Overview

In [44]:
# read in datasets

atbats = pd.read_csv('Data/atbats.csv')
pitches = pd.read_csv('Data/pitches.csv')
atbats19 = pd.read_csv('Data/2019_atbats.csv')
pitches19 = pd.read_csv('Data/2019_pitches.csv')

In [45]:
atbats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740389 entries, 0 to 740388
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   ab_id       740389 non-null  int64 
 1   batter_id   740389 non-null  int64 
 2   event       740389 non-null  object
 3   g_id        740389 non-null  int64 
 4   inning      740389 non-null  int64 
 5   o           740389 non-null  int64 
 6   p_score     740389 non-null  int64 
 7   p_throws    740389 non-null  object
 8   pitcher_id  740389 non-null  int64 
 9   stand       740389 non-null  object
 10  top         740389 non-null  bool  
dtypes: bool(1), int64(7), object(3)
memory usage: 57.2+ MB


In [46]:
atbats19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185245 entries, 0 to 185244
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   inning      185245 non-null  int64 
 1   top         185245 non-null  int64 
 2   ab_id       185245 non-null  int64 
 3   g_id        185245 non-null  int64 
 4   p_score     185245 non-null  int64 
 5   batter_id   185245 non-null  int64 
 6   pitcher_id  185245 non-null  int64 
 7   stand       185245 non-null  object
 8   p_throws    185245 non-null  object
 9   event       185245 non-null  object
 10  o           185245 non-null  int64 
dtypes: int64(8), object(3)
memory usage: 15.5+ MB


In [47]:
# rearrange columns in atbats to match column order of atbats19

atbats = atbats[['inning', 'top', 'ab_id', 'g_id', 'p_score', 'batter_id', 'pitcher_id', 'stand', 'p_throws', 'event', 'o']]
atbats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740389 entries, 0 to 740388
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   inning      740389 non-null  int64 
 1   top         740389 non-null  bool  
 2   ab_id       740389 non-null  int64 
 3   g_id        740389 non-null  int64 
 4   p_score     740389 non-null  int64 
 5   batter_id   740389 non-null  int64 
 6   pitcher_id  740389 non-null  int64 
 7   stand       740389 non-null  object
 8   p_throws    740389 non-null  object
 9   event       740389 non-null  object
 10  o           740389 non-null  int64 
dtypes: bool(1), int64(7), object(3)
memory usage: 57.2+ MB


In [51]:
# concatenate the two atbats Dataframes

atbats_all = pd.concat([atbats, atbats19], ignore_index=True, sort=False)
atbats_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 925634 entries, 0 to 925633
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   inning      925634 non-null  int64 
 1   top         925634 non-null  int64 
 2   ab_id       925634 non-null  int64 
 3   g_id        925634 non-null  int64 
 4   p_score     925634 non-null  int64 
 5   batter_id   925634 non-null  int64 
 6   pitcher_id  925634 non-null  int64 
 7   stand       925634 non-null  object
 8   p_throws    925634 non-null  object
 9   event       925634 non-null  object
 10  o           925634 non-null  int64 
dtypes: int64(8), object(3)
memory usage: 77.7+ MB


In [39]:
atbats_all = pd.merge(atbats19, atbats, how='outer', on='ab_id')
atbats_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740389 entries, 0 to 740388
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   inning      740389 non-null  int64 
 1   top         740389 non-null  bool  
 2   ab_id       740389 non-null  int64 
 3   g_id        740389 non-null  int64 
 4   p_score     740389 non-null  int64 
 5   batter_id   740389 non-null  int64 
 6   pitcher_id  740389 non-null  int64 
 7   stand       740389 non-null  object
 8   p_throws    740389 non-null  object
 9   event       740389 non-null  object
 10  o           740389 non-null  int64 
dtypes: bool(1), int64(7), object(3)
memory usage: 57.2+ MB


In [54]:
pitches.rename(columns = {'px': 'horiz_loc', 'pz': 'vert_loc', 'pfx_x': 'horiz_move', 'pfx_z': 'vert_move'}, inplace=True)

In [55]:
pitches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2867154 entries, 0 to 2867153
Data columns (total 40 columns):
 #   Column           Dtype  
---  ------           -----  
 0   horiz_loc        float64
 1   vert_loc         float64
 2   start_speed      float64
 3   end_speed        float64
 4   spin_rate        float64
 5   spin_dir         float64
 6   break_angle      float64
 7   break_length     float64
 8   break_y          float64
 9   ax               float64
 10  ay               float64
 11  az               float64
 12  sz_bot           float64
 13  sz_top           float64
 14  type_confidence  float64
 15  vx0              float64
 16  vy0              float64
 17  vz0              float64
 18  x                float64
 19  x0               float64
 20  y                float64
 21  y0               float64
 22  z0               float64
 23  horiz_move       float64
 24  vert_move        float64
 25  nasty            float64
 26  zone             float64
 27  code        

In [56]:
pitches19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728790 entries, 0 to 728789
Data columns (total 40 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   horiz_loc        722161 non-null  float64
 1   vert_loc         722161 non-null  float64
 2   start_speed      722161 non-null  float64
 3   end_speed        722161 non-null  float64
 4   spin_rate        722161 non-null  object 
 5   spin_dir         722161 non-null  object 
 6   break_angle      722161 non-null  float64
 7   break_length     722161 non-null  float64
 8   break_y          722161 non-null  float64
 9   ax               722161 non-null  float64
 10  ay               722161 non-null  float64
 11  az               722161 non-null  float64
 12  sz_bot           728790 non-null  float64
 13  sz_top           728790 non-null  float64
 14  type_confidence  722161 non-null  object 
 15  vx0              722161 non-null  float64
 16  vy0              722161 non-null  floa

In [57]:
# concatenate the two pitches Dataframes

pitches_all = pd.concat([pitches, pitches19], ignore_index=True, sort=False)
pitches_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3595944 entries, 0 to 3595943
Data columns (total 40 columns):
 #   Column           Dtype  
---  ------           -----  
 0   horiz_loc        float64
 1   vert_loc         float64
 2   start_speed      float64
 3   end_speed        float64
 4   spin_rate        object 
 5   spin_dir         object 
 6   break_angle      float64
 7   break_length     float64
 8   break_y          float64
 9   ax               float64
 10  ay               float64
 11  az               float64
 12  sz_bot           float64
 13  sz_top           float64
 14  type_confidence  object 
 15  vx0              float64
 16  vy0              float64
 17  vz0              float64
 18  x                object 
 19  x0               float64
 20  y                object 
 21  y0               float64
 22  z0               float64
 23  horiz_move       float64
 24  vert_move        float64
 25  nasty            float64
 26  zone             object 
 27  code        

In [34]:
# drop columns in pitches DataFrame that have no data

#pitches19 = pitches19.drop(['spin_rate', 'spin_dir', 'type_confidence', 'zone', 'nasty'], axis=1)

In [67]:
pitches_all['type'].value_counts()

S     1384615
B     1289027
X      592725
F      128929
C      119334
D       28164
*B      17505
E       16946
T        6447
W        5552
V        2611
H        1963
L        1684
M         353
P          60
O          29
Name: type, dtype: int64

in pitches_all, need to delete rows with duplicate ab_id -- only keep the ab_id with highest pitch_num